## Instalación de dependencias y recuperación de funciones RAG y LLM

In [1]:
# Instalamos las dependencias. Podemos observar que saltan conflictos de dependencias cruzadas pero la instalación clave es correcta y se puede continuar con el código.

!pip install -q gradio==4.44.0 sentence-transformers faiss-cpu python-dotenv
!pip install -q "websockets>=13,<15" # Útil para comunicación en tiempo real.
!pip install -q llama-cpp-python # Para emplear llama.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 12.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.65 requires websockets>=13.0, but you have websockets 12.0 which is incompatible.
google-genai 1.34.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 12.0 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 12.0 which is incompatible.
google-adk 1.13.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 12.0 which is incompatible.
   ━━━━━

In [2]:
# Realizamos conexión con google drive para poder ejecutar los notebooks.

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# En esta parte del código se realiza lo siguiente:
# Si tiene GPU utilizará la GPU. En caso contrario, se utiliza la CPU sin proporcionar errores.

import torch
from sentence_transformers import SentenceTransformer

try:
    # Detecta si hay GPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Usando dispositivo: {device}")

    # Carga el modelo en GPU si está disponible
    model_emb = SentenceTransformer("all-MiniLM-L6-v2", device=device)

    # Test rápido
    _ = model_emb.encode(["prueba"])
    print("Modelo cargado correctamente empleando GPU.")

except Exception as e:
    print("Error en GPU, reasignando a CPU…", e)
    model_emb = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")

Usando dispositivo: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo cargado correctamente empleando GPU.


In [4]:
# Convertimos los .ipynb a .py.

BASE = "/content/drive/MyDrive/forma_dividida"

!jupyter nbconvert --to python "{BASE}/Fase_1_limpieza_chunks.ipynb" --output "Fase_1_limpieza_chunks" --output-dir "{BASE}"
!jupyter nbconvert --to python "{BASE}/Fase_2_embeddings_local.ipynb" --output "Fase_2_embeddings_local.ipynb" --output-dir "{BASE}"

print("Notebooks convertidos a .py correctamente.")



[NbConvertApp] Converting notebook /content/drive/MyDrive/forma_dividida/Fase_1_limpieza_chunks.ipynb to python
[NbConvertApp] Writing 12118 bytes to /content/drive/MyDrive/forma_dividida/Fase_1_limpieza_chunks.py
[NbConvertApp] Converting notebook /content/drive/MyDrive/forma_dividida/Fase_2_embeddings_local.ipynb to python
[NbConvertApp] Writing 35644 bytes to /content/drive/MyDrive/forma_dividida/Fase_2_embeddings_local.ipynb.py
Notebooks convertidos a .py correctamente.


In [5]:
import os, sys, re, types, inspect
from pathlib import Path
import importlib.util

BASE = "/content/drive/MyDrive/forma_dividida"
src_path = Path(BASE) / "Fase_2_embeddings_local.ipynb.py" # Cargamos el fichero de origen.
tmp_path = Path("/content/Fase_2_embeddings_cpu_stable.py") # Ruta donde escribimos la versión temporal.
assert src_path.exists(), f"No existe: {src_path}"

# Leemos el fichero

print("Leyendo y parcheando a CPU…")
with open(src_path, "r", encoding="utf-8") as f:
    src = f.read()

# Parche mínimo y seguro a CPU
# 1) device='cuda' / "cuda" → 'cpu'
src = re.sub(r"device\s*=\s*['\"]cuda['\"]", "device='cpu'", src)
src = re.sub(r"device\s*=\s*\"cuda\"", "device='cpu'", src)

# 2) n_gpu_layers = X → 0 (si aparece)
src = re.sub(r"n_gpu_layers\s*=\s*\d+", "n_gpu_layers=0", src)

# 3) patrones típicos que fuerzan CUDA en sentence-transformers
src = re.sub(r"\.to\(\s*['\"]cuda['\"]\s*\)", ".to('cpu')", src)

# Guardamos el código parcheado (temporal)

with open(tmp_path, "w", encoding="utf-8") as f:
    f.write(src)
print(f"Guardado temporal parcheado en: {tmp_path}")

# Importación limpia del módulo parcheado
spec = importlib.util.spec_from_file_location("Fase_2_embeddings_cpu_stable", str(tmp_path))
F2 = importlib.util.module_from_spec(spec)
sys.modules["Fase_2_embeddings_cpu_stable"] = F2
spec.loader.exec_module(F2)
print("Módulo F2 importado.")

# Lista de nombres definidos en el módulo que no empiezan con _. Esto sirve para ver que funciones o variables hay disponibles
public = [n for n in dir(F2) if not n.startswith("_")]
print("Símbolos públicos detectados en F2:")
print(public)

# Detectar rag_pipeline
rag_pipeline = None
for candidate in ["rag_pipeline", "query_rag", "pipeline_rag", "pipeline", "retrieve_context", "retrieve", "buscar"]:
    if hasattr(F2, candidate) and isinstance(getattr(F2, candidate), types.FunctionType):
        rag_pipeline = getattr(F2, candidate)
        print(f"rag_pipeline = F2.{candidate}")
        break

if rag_pipeline is None:
    # Heurística: primera función que acepte (query, k, model/embedding)
    for n in public:
        obj = getattr(F2, n)
        if isinstance(obj, types.FunctionType):
            sig = str(inspect.signature(obj))
            if "k" in sig and ("model" in sig or "embedding" in sig or "model_obj" in sig):
                rag_pipeline = obj
                print(f"rag_pipeline (heurística) = F2.{n}")
                break
    assert rag_pipeline is not None, "No encuentro una función tipo rag_pipeline en F2."

# Detectar model_emb
# En vez de usar la clase, instanciamos el modelo en GPU. Si falla, lo carga en CPU.
from sentence_transformers import SentenceTransformer
try:
    model_emb = SentenceTransformer('all-MiniLM-L6-v2', device="cuda")
    print("model_emb cargado en GPU (all-MiniLM-L6-v2).")
except Exception as e:
    print("No se pudo cargar en GPU, se usa CPU:", e)
    model_emb = SentenceTransformer('all-MiniLM-L6-v2', device="cpu")

# Detectar get_llm_answer (función que genera la respuesta)
get_llm_answer = None
for candidate in ["get_llm_answer","llm_answer","get_answer","answer_query","ask_llm","ask","query_answer","generate_answer"]:
    if hasattr(F2, candidate) and isinstance(getattr(F2, candidate), types.FunctionType):
        get_llm_answer = getattr(F2, candidate)
        print(f"get_llm_answer = F2.{candidate}")
        break

if get_llm_answer is None:
    # Heurística: función que reciba (pregunta, rag_pipeline, embedding_model, k, temperature, max_tokens)
    for n in public:
        obj = getattr(F2, n)
        if isinstance(obj, types.FunctionType):
            params = list(inspect.signature(obj).parameters)
            joined = " ".join(params).lower()
            if ("rag" in joined or "pipeline" in joined) and ("embed" in joined or "embedding" in joined):
                if any("temp" in p for p in params) and any(p=="k" or "k" in p for p in params):
                    get_llm_answer = obj
                    print(f"get_llm_answer (heurística) = F2.{n}")
                    break
    assert get_llm_answer is not None, "No encuentro la función de respuesta del LLM en F2."

# Prueba de que los embeddings están operativos. Captura errores y los reporta.
try:
    _ = model_emb.encode(["test"])
    print("Embeddings operativos (encode).")
except Exception as e:
    print("Embeddings dieron error al probar encode():", e)

print("\n Aliases listos para la FASE 3: rag_pipeline, model_emb, get_llm_answer\n")


Leyendo y parcheando a CPU…
Guardado temporal parcheado en: /content/Fase_2_embeddings_cpu_stable.py
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Batches:   0%|          | 0/207 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluación obtenida


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluación obtenida
Guardado: {'model': 'local', 'k': 8, 'temperature': 0.2, 'max_tokens': 512, 'BLEU': np.float64(0.02875973682251983), 'METEOR': np.float64(0.14682074585719118), 'ROUGE-L': np.float64(0.1229393115942029), 'Embedding_Cosine': np.float32(0.7220113)}

CSV guardado en: /content/drive/MyDrive/resultados_csv/llm_eval_results_20250916_182005.csv
Módulo F2 importado.
Símbolos públicos detectados en F2:
['AutoModelForCausalLM', 'AutoTokenizer', 'CrossEncoder', 'Llama', 'SentenceTransformer', 'SmoothingFunction', 'answer', 'build_context', 'chunks_para_rag', 'chunks_para_rag_str', 'cosine_similarity', 'csv', 'csv_columns', 'csv_file', 'datetime', 'dim', 'drive', 'e', 'embedding_model', 'embeddings', 'encode', 'eval_data_llm', 'evaluate_llm_answers', 'f', 'faiss', 'get_embedding', 'get_llm_answer', 'get_llm_answers_from_rag', 'hf_hub_download', 'index', 'itertools', 'k', 'k_values', 'llm', 'llm_answers', 'login', 'max_tok', 'max_tokens_list', 'metrics', 'model_emb', 'model_name'

In [ ]:
# =========================================================
# MeRenta — Asistente de Declaración de la Renta
# =========================================================

import gradio as gr   # Biblioteca para crear interfaces web interactivas
import traceback      # Para capturar y mostrar errores de forma detallada


# Parámetros internos - configuración por defecto del LLM

DEFAULT_K = 4
DEFAULT_TEMPERATURE = 0.01
DEFAULT_MAX_TOKENS = 128
DEFAULT_TOP_P = 0.8

# Funciones

def get_llm_answer_wrapped(query: str):
    """
    Función envoltorio que valida la consulta del usuario y llama al pipeline RAG.
    - Comprueba que la query no esté vacía.
    - Ejecuta la función get_llm_answer con los parámetros predefinidos.
    - Captura posibles excepciones y devuelve un mensaje de error legible.
    """
    try:
        # Validar que la consulta no sea vacía
        if not query or not query.strip():
            return "Por favor, escribe una consulta clara sobre tu declaración de la renta."

        # Llamada al LLM con el pipeline RAG y parámetros por defecto
        return get_llm_answer(
            query.strip(),
            rag_pipeline,
            embedding_model=model_emb,
            k=DEFAULT_K,
            temperature=DEFAULT_TEMPERATURE,
            max_tokens=DEFAULT_MAX_TOKENS,
            top_p=DEFAULT_TOP_P,
        )
    except Exception:
        # Si ocurre un error, se muestra el traceback detallado
        return "Se produjo un error al generar la respuesta:\n\n" + traceback.format_exc()


def interface_fn(query: str):
    """
    Función puente para Gradio.
    - Recibe la consulta del usuario desde la interfaz.
    - Llama a get_llm_answer_wrapped.
    - Convierte el resultado en string (Gradio lo requiere).
    """
    return str(get_llm_answer_wrapped(query))


def clear_all():
    """
    Función auxiliar para limpiar los campos de entrada y salida de la interfaz.
    - Devuelve dos strings vacíos que corresponden al input y al output.
    """
    return "", ""


# Estilos CSS personalizados para la interfaz Gradio

custom_css = """
:root {
  --brand:#0A66C2;
  --brand-dark:#084c8d;
  --bg:#f9fafc;
  --text:#2c3e50;
  --muted:#606f7b;
  --card-bg:#ffffff;
  --card-border:#dce1e8;
}
.gradio-container {
  font-family: 'Inter', ui-sans-serif, system-ui;
  background: var(--bg);
  color: var(--text);
  max-width: 100% !important;
  padding: 0 !important;
}
.mr-hero {
  text-align:center;
  padding: 50px 20px 20px;
  background: linear-gradient(135deg, #eaf3ff 0%, #ffffff 90%);
}
.mr-title {
  font-size: clamp(36px, 5vw, 52px);
  font-weight: 800;
  color: var(--brand);
  margin-bottom: 10px;
}
.mr-subtitle {
  max-width: 900px;
  margin: 10px auto 0;
  font-size: clamp(16px, 2vw, 20px);
  color: var(--muted);
}
.mr-subtitle b {
  background-color: rgba(10,102,194,0.1);
  color: var(--brand);
  padding: 2px 6px;
  border-radius: 6px;
}
.mr-card {
  margin: 26px auto;
  max-width: 900px;
  background: var(--card-bg);
  border: 1px solid var(--card-border);
  border-radius: 12px;
  box-shadow: 0 8px 24px rgba(0,0,0,0.05);
  padding: 24px;
}
.gr-button.primary {
  background: var(--brand);
  border: none;
  color: #fff;
  font-weight: 700;
  border-radius: 8px;
  padding: 12px 18px;
  font-size: 16px;
  transition: background 0.2s ease-in-out;
}
.gr-button.primary:hover {
  background: var(--brand-dark);
}
.gr-button.secondary {
  background: #f1f3f6;
  border: 1px solid #ccd2d8;
  color: var(--text);
  border-radius: 8px;
}
textarea, input {
  caret-color: var(--brand);
}
.mr-footer {
  text-align:center;
  color: var(--muted);
  font-size: 13px;
  margin: 20px auto 30px;
  max-width: 980px;
}
.mr-disclaimer {
  margin-top: 18px;
  color: #4f5d75;
  font-size: 13px;
  line-height:1.5;
  border-left: 4px solid rgba(10,102,194,0.4);
  padding-left: 10px;
  background: #f8fbff;
}
"""

# Construcción de la interfaz

with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:

    # Sección superior (hero)
    gr.HTML(
        """
        <section class="mr-hero">
          <h1 class="mr-title">MeRenta</h1>
          <p class="mr-subtitle">
            Tu asistente inteligente para la <b>declaración de la renta en España</b>.
            Responde tus dudas con información clara y actualizada.
          </p>
        </section>
        """
    )

    # Tarjeta principal con input y output
    with gr.Group(elem_classes="mr-card"):

        # Entrada de texto para la consulta
        with gr.Row():
            query_in = gr.Textbox(
                label="Consulta tributaria",
                placeholder="Ej.: ¿Qué deducciones puedo aplicar si trabajo desde casa?",
                lines=3,
                autofocus=True,
            )

        # Botones de acción
        with gr.Row():
            run_btn = gr.Button("Generar respuesta", variant="primary", scale=2)
            clear_btn = gr.Button("Limpiar", variant="secondary")

        # Salida en formato Markdown
        with gr.Row():
            output_md = gr.Markdown(label="Respuesta", value="")

        # Aviso legal en la parte inferior de la tarjeta
        gr.HTML(
            """
            <div class="mr-disclaimer">
              <strong>Aviso:</strong> La información proporcionada por MeRenta es orientativa y no tiene carácter vinculante frente a la Agencia Tributaria.
              Consulta siempre la normativa oficial o asesórate con un profesional para casos específicos.
            </div>
            """
        )

    # Footer con créditos
    gr.HTML(
        """
        <div class="mr-footer">
          © MeRenta — Herramienta informativa. Diseño orientado a claridad y confianza.
        </div>
        """
    )

    # Conectar botón "Generar respuesta" con la función interface_fn

    run_btn.click(interface_fn, inputs=[query_in], outputs=[output_md])

    # Conectar botón "Limpiar" con la función clear_all

    clear_btn.click(fn=clear_all, inputs=None, outputs=[query_in, output_md])

# Lanzamiento de la app
if __name__ == "__main__":
    demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b383e957a957c5a810.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
